In [1]:
import gymnasium as gym
from gymnasium.spaces import Discrete, Box, Sequence, Dict
import numpy as np
import pandas as pd
from pandarallel import pandarallel
from src.commons import methods
import statistics

from src.rl_environments.env_impr3 import EnvImpr3
from src.model_approaches.analytical.oracle_fix_n import OracleFix_n
from src.model_approaches.analytical.oracle_var_n import OracleVar_n
from src.model_approaches.analytical.pto_fix_n import PTO_Fix_n
from src.model_approaches.analytical.pto_var_n import PTO_Var_n
from src.model_approaches.rl.ppo import PPO

from src.commons import generate_data, methods
from src.commons.constants import AlphaEstimator, TravelTimeDist
import os
import pickle
from pathlib import Path

from collections import OrderedDict

## Train Config

In [2]:
PARAM_ESTIMATOR = AlphaEstimator.MAX_LIKELI

TRAIN_CONFIG = {
    'alpha_range': np.arange(0.1, 7.0, 0.1).tolist(),
    'beta_range': np.arange(0.1, 5.0, 0.1).tolist(),
    'h_range': [0.5],
    'c_range': [25],
    'total': list(range(5, 30)),
    'travel_time': TravelTimeDist.UNIFORM,
    'param_estimator': PARAM_ESTIMATOR,
}

PPO_TIME_STEPS = 2_000_000
PPO_ENV_CLASS = EnvImpr3


## Model Options

In [3]:
oracle_models = [OracleVar_n(), OracleFix_n(n=3), OracleFix_n(n=5)]
user_models = [PPO(EnvClass=EnvImpr3, config=TRAIN_CONFIG, timesteps=PPO_TIME_STEPS),
               PTO_Var_n(param_estimator=PARAM_ESTIMATOR),
               PTO_Fix_n(n=3, param_estimator=PARAM_ESTIMATOR),
               PTO_Fix_n(n=5, param_estimator=PARAM_ESTIMATOR)]

cost_results_df = pd.DataFrame(columns=['Test Name', 'PPO', 'PTO_Var_n', 'Oracle_Var_n', 'PTO_Fix_n_3', 'Oracle_Fix_n_3', 'PTO_Fix_n_5', 'Oracle_Fix_n_5'])
observed_n_results_df = pd.DataFrame(columns=['Test Name', 'PPO', 'PTO_Var_n', 'Oracle_Var_n'])

[INFO] Training PPO model...
Saving to: /Users/preetkaria/PycharmProjects/when_to_leave/models/PPO_EnvImpr3_c49ab_2M.zip
[INFO] Launching training with 8 parallel environments
Using cpu device
[INFO] Training PPO for 2000000 timesteps...
---------------------------------━━━━━━━━━━ 16,264/2,000,000  [ 0:00:40 < 1:20:47 , 409 it/s ]
| rollout/           |          |
|    ep_len_mean     | 3.85     |
|    ep_rew_mean     | -28      |
| time/              |          |
|    fps             | 404      |
|    iterations      | 1        |
|    time_elapsed    | 40       |
|    total_timesteps | 16384    |
---------------------------------
-----------------------------------------━━ 32,632/2,000,000  [ 0:01:20 < 1:13:47 , 444 it/s ]
| rollout/                |             |
|    ep_len_mean          | 4.13        |
|    ep_rew_mean          | -27.4       |
| time/                   |             |
|    fps                  | 408         |
|    iterations           | 2           |
|    time_elap

## Common Methods

In [4]:
def getResultColumnName(column: str, columns_to_visualize: list, type: str) -> str:
    for col in columns_to_visualize:
        if type in col and column in col:
            return col
    print(f"No matching column found for {column} with type {type}")

def testFlow(test_df_path: str, models: list):
    # Load test data
    test_df = pd.DataFrame()
    with open(test_df_path, 'rb') as f:
        test_df =pickle.load(f)
        
    # Evaluate models
    
    columns_to_visualize = []
    for model in models:
        print(f'Evaluating {model.prediction_keys()}')
        columns_to_visualize += model.prediction_keys()
        
        new_cols = test_df.parallel_apply(lambda row: model.predict(row, override=False), result_type='expand', axis=1)
        for i, col_name in enumerate(model.prediction_keys()):
            test_df[col_name] = new_cols[i]
            print(col_name, test_df[col_name].mean())
            
    # Save the test df with predictions
    test_df_path = Path(test_df_path)
    test_df.to_csv(test_df_path.with_suffix('.csv'), index=False)
    test_df.to_pickle(test_df_path)
    
    return test_df, columns_to_visualize

def evaluateResults(test_df: pd.DataFrame, columns_to_visualize: list, test_name: str, 
                    cost_results_df: pd.DataFrame, observed_n_results_df: pd.DataFrame):
    means = test_df[columns_to_visualize].mean()
    cost_results_row = {'Test Name': test_name}
    for cost_column in cost_results_df.columns:
        if cost_column == 'Test Name':
            continue
        mean_col = getResultColumnName(cost_column, columns_to_visualize, 'cost')
        cost_results_row[cost_column] = means.get(mean_col)
    cost_results_df = pd.concat([cost_results_df, pd.DataFrame([cost_results_row])], ignore_index=True)
    
    observed_n_results_rows = {'Test Name': test_name}
    for observed_n_column in observed_n_results_df.columns:
        if observed_n_column == 'Test Name':
            continue
        mean_col = getResultColumnName(observed_n_column, columns_to_visualize, 'observed_n')
        observed_n_results_rows[observed_n_column] = means.get(mean_col)
    observed_n_results_df = pd.concat([observed_n_results_df, pd.DataFrame([observed_n_results_rows])], ignore_index=True)
    
    return cost_results_df, observed_n_results_df

## Test on Simulated Data

In [5]:
base_sim_test = OrderedDict()
base_sim_test['low alpha from 0.1 to 1.0'] = {
    'alpha_range': np.arange(0.1, 1.0, 0.1).tolist(),
    'beta_range': np.arange(1.1, 2.0, 0.1).tolist(),
    'h_range': [0.5],
    'c_range': [25],
    'total': list(range(10, 30)),
    'travel_time': TravelTimeDist.UNIFORM,
    'param_estimator': PARAM_ESTIMATOR,
}

base_sim_test['medium alpha from 1.0 to 3.0'] = {
    'alpha_range': np.arange(1.0, 3.0, 0.1).tolist(),
    'beta_range': np.arange(1.1, 2.0, 0.1).tolist(),
    'h_range': [0.5],
    'c_range': [25],
    'total': list(range(10, 30)),
    'travel_time': TravelTimeDist.UNIFORM,
    'param_estimator': PARAM_ESTIMATOR,
}

base_sim_test['high alpha from 3.0 to 5.0'] = {
    'alpha_range': np.arange(3.0, 5.0, 0.1).tolist(),
    'beta_range': np.arange(1.1, 2.0, 0.1).tolist(),
    'h_range': [0.5],
    'c_range': [25],
    'total': list(range(10, 30)),
    'travel_time': TravelTimeDist.UNIFORM,
    'param_estimator': PARAM_ESTIMATOR,
}

base_sim_test['high beta from 3 to 5'] = {
    'alpha_range': np.arange(0.1, 3.0, 0.1).tolist(),
    'beta_range': np.arange(3.0, 5.0, 0.1).tolist(),
    'h_range': [0.5],
    'c_range': [25],
    'total': list(range(10, 30)),
    'travel_time': TravelTimeDist.UNIFORM,
    'param_estimator': PARAM_ESTIMATOR,
}
sim_tests = OrderedDict()
for travel_time in TravelTimeDist:
    for test_name, config in base_sim_test.items():
        new_test_name = f"{test_name} with {travel_time.name.lower()} travel time"
        new_config = config.copy()
        new_config['travel_time'] = travel_time
        sim_tests[new_test_name] = new_config


In [6]:
for test_name, config in sim_tests.items():
    print(f'Running test: {test_name}')
    
    test_df_path = generate_data.generate(config=config, row_count=10_000)
    
    pandarallel.initialize(progress_bar=True)
        
    test_df = pd.DataFrame()
    with open(test_df_path, 'rb') as f:
        test_df =pickle.load(f)
        
    # Evaluate models
    
    columns_to_visualize = []
    for model in user_models + oracle_models:
        print(f'Evaluating {model.prediction_keys()}')
        columns_to_visualize += model.prediction_keys()
        
        new_cols = test_df.parallel_apply(lambda row: model.predict(row, override=False), result_type='expand', axis=1)
        for i, col_name in enumerate(model.prediction_keys()):
            test_df[col_name] = new_cols[i]
            print(col_name, test_df[col_name].mean())
            
    # Save the test df with predictions
    test_df_path = Path(test_df_path)
    test_df.to_csv(test_df_path.with_suffix('.csv'), index=False)
    test_df.to_pickle(test_df_path)

    cost_results_df, observed_n_results_df = evaluateResults(test_df, columns_to_visualize, test_name, cost_results_df, observed_n_results_df)
    
    
        
        
    
    

Running test: low alpha from 0.1 to 1.0 with uniform travel time
[INFO] Generating new dataset:
/Users/preetkaria/PycharmProjects/when_to_leave/data/9f469_10K

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Evaluating ['cost_PPO_EnvImpr3_c49ab_2M', 'observed_n_PPO_EnvImpr3_c49ab_2M']


cost_PPO_EnvImpr3_c49ab_2M 3.6373386900797544
observed_n_PPO_EnvImpr3_c49ab_2M 5.0915
Evaluating ['cost_PTO_Var_n', 'observed_n_PTO_Var_n']


cost_PTO_Var_n 3.783314253926137
observed_n_PTO_Var_n 4.6449
Evaluating ['cost_PTO_Fix_n_3']


cost_PTO_Fix_n_3 6.67444801218472
Evaluating ['cost_PTO_Fix_n_5']


cost_PTO_Fix_n_5 6.575804684353
Evaluating ['cost_Oracle_Var_n', 'observed_n_Oracle_Var_n']


...

cost_PTO_Fix_n_3 13.544977149916333
Evaluating ['cost_PTO_Fix_n_5']


cost_PTO_Fix_n_5 13.59653249985598
Evaluating ['cost_Oracle_Var_n', 'observed_n_Oracle_Var_n']


cost_Oracle_Var_n 9.648409532024845
observed_n_Oracle_Var_n 4.1128
Evaluating ['cost_Oracle_Fix_n_3']


cost_Oracle_Fix_n_3 10.177716554396914
Evaluating ['cost_Oracle_Fix_n_5']


cost_Oracle_Fix_n_5 12.17840128372822


## Test Prof Data

In [7]:
test_df_path = Path(methods.file_path('prof_full_data.pkl', dir_name='data/prof_data_test/'))
test_name = "Prof Data Test"

pandarallel.initialize(progress_bar=True)
        
test_df = pd.DataFrame()
with open(test_df_path, 'rb') as f:
    test_df =pickle.load(f)


# Evaluate models

columns_to_visualize = []
for model in user_models:
    print(f'Evaluating {model.prediction_keys()}')
    columns_to_visualize += model.prediction_keys()
    
    new_cols = test_df.parallel_apply(lambda row: model.predict(row, override=False), result_type='expand', axis=1)
    for i, col_name in enumerate(model.prediction_keys()):
        test_df[col_name] = new_cols[i]
        print(col_name, test_df[col_name].mean())
        
# Save the test df with predictions
test_df_path = Path(test_df_path)
test_df.to_csv(test_df_path.with_suffix('.csv'), index=False)
test_df.to_pickle(test_df_path)

cost_results_df, observed_n_results_df = evaluateResults(test_df, columns_to_visualize, test_name, cost_results_df, observed_n_results_df)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Evaluating ['cost_PPO_EnvImpr3_c49ab_2M', 'observed_n_PPO_EnvImpr3_c49ab_2M']


Process ForkPoolWorker-760:
Process ForkPoolWorker-764:
Process ForkPoolWorker-758:
Process ForkPoolWorker-761:
Process ForkPoolWorker-762:
Process ForkPoolWorker-765:
Process ForkPoolWorker-759:
Process ForkPoolWorker-763:
Traceback (most recent call last):
  File "/Users/preetkaria/miniconda3/envs/when_to_leave/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/preetkaria/miniconda3/envs/when_to_leave/lib/python3.13/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/preetkaria/miniconda3/envs/when_to_leave/lib/python3.13/site-packages/sympy/core/assumptions.py", line 499, in getit
 

KeyboardInterrupt: 

## Test Bicycle Data

In [ ]:
test_df_path = Path(methods.file_path('bicycle_data.pkl', dir_name='data/non_gamma_bicyle_test'))
test_name = "Bicycle Data Test"

pandarallel.initialize(progress_bar=True)
        
test_df = pd.DataFrame()
with open(test_df_path, 'rb') as f:
    test_df =pickle.load(f)
    
# Evaluate models

columns_to_visualize = []
for model in user_models:
    print(f'Evaluating {model.prediction_keys()}')
    columns_to_visualize += model.prediction_keys()
    
    new_cols = test_df.parallel_apply(lambda row: model.predict(row, override=False), result_type='expand', axis=1)
    for i, col_name in enumerate(model.prediction_keys()):
        test_df[col_name] = new_cols[i]
        print(col_name, test_df[col_name].mean())
        
# Save the test df with predictions
test_df_path = Path(test_df_path)
test_df.to_csv(test_df_path.with_suffix('.csv'), index=False)
test_df.to_pickle(test_df_path)

cost_results_df, observed_n_results_df = evaluateResults(test_df, columns_to_visualize, test_name, cost_results_df, observed_n_results_df)

In [ ]:
display(cost_results_df.head())
display(observed_n_results_df.head())
cost_results_df.to_csv(methods.file_path('finla_test_cost_results.csv', dir_name='results'), index=False)
observed_n_results_df.to_csv(methods.file_path('finla_test_observed_n_results.csv', dir_name='results'), index=False)

## Average Cost Comparison Plot

In [ ]:
method_map = {'PPO': 'RL', 'PTO_Var_n': 'PTO (var n)', 'Oracle_Var_n': 'Oracle (var n)',
              'PTO_Fix_n_3': 'PTO (n = 3)', 'Oracle_Fix_n_3': 'Oracle (n = 3)',
              'PTO_Fix_n_5': 'PTO (n = 5)', 'Oracle_Fix_n_5': 'Oracle (n = 5)'}
color_map = {'PPO': '#377eb8', 'PTO_Var_n': '#ff7f00', 'Oracle_Var_n': '#4daf4a',
              'PTO_Fix_n_3': "#ffad5bff", 'Oracle_Fix_n_3': "#77ae75ff",
              'PTO_Fix_n_5': "#ffad5bff", 'Oracle_Fix_n_5': "#77ae75ff"}
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import math
from matplotlib.ticker import MaxNLocator

sns.set_theme(style='whitegrid', context='paper', font_scale=1.5, rc={"axes.labelweight": "bold"})


for row in cost_results_df.itertuples():
    
    test_name = row._1
    methods_list = list(cost_results_df.columns)[1:]
    
    if test_name == "Bicycle Data Test" or test_name == "Prof Data Test":
        methods_list = ['PPO', 'PTO_Var_n', 'PTO_Fix_n_3', 'PTO_Fix_n_5']
        
    method_list = [m for m in methods_list if not math.isnan(getattr(row, m))]
    
    costs = [getattr(row, col) for col in methods_list]
    
    
    plt.figure(figsize=(10, 6))
    
    short_labels = [method_map[m] for m in methods_list]
    
    colors = [color_map[m] for m in methods_list]

   
    ax = sns.barplot(x=short_labels, y=costs, palette=colors, edgecolor='black', linewidth=0.6)
    ax.set_ylabel('Average Cost')
    ax.set_xlabel('Method')
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.grid(False)
    
    sns.despine()
   
   
    for i, v in enumerate(costs):
        ax.text(i, v + 0.5, f"{v:.3f}", ha='center', va='bottom')
    
    plt.tight_layout()
    
    config = sim_tests.get(test_name)
   
    file_name = f'cost_comparison_{test_name.replace(" ", "_").lower()}'
    plt.savefig(methods.file_path(f'{file_name}.png', dir_name='figures'))
    print(f'Saved figure to {methods.file_path(f"{file_name}.png", dir_name="figures")}')
    json.dump(config, open(methods.file_path(f'{file_name}.json', dir_name='figures'), 'w'), indent=4)
    plt.show()

## Averge Cost Compiled

In [ ]:
# obersved n compiled plot

# Preserve 'Test Name' so it can be used on x-axis
cost_results_df_melted = cost_results_df.melt(
	id_vars='Test Name',
	var_name='Method',
	value_name='Average Cost'
)
display(cost_results_df_melted.head())
cost_results_df_melted = cost_results_df_melted[
    cost_results_df_melted['Test Name'].str.contains('with', na=False)
    & ~cost_results_df_melted['Method'].str.contains('fix', case=False, na=False)
]
sns.set_theme(style='whitegrid', context='paper', font_scale=1.5, rc={"axes.labelweight": "bold"})
plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=cost_results_df_melted,
    x='Test Name',
    y='Average Cost',
    hue='Method',
    palette=color_map,
    edgecolor='black',
    linewidth=0.6
)
ax.set_ylabel('Average Cost', fontdict={'size': 14})
ax.set_xlabel('Simulated Gamma Tests', fontdict={'size': 14})
ax.set_xticklabels([
    r'low $\alpha$',
    r'mid $\alpha$',
    r'high $\alpha$',
    r'high $\beta$',  
] * 4)

# highlight groups: 1-4 uniform, 5-8 low, 9-12 high
group_spans = [
    (-0.5, 3.5, 'Uniform Travel Time'),
    (3.5, 7.5, 'Low Travel Time'),
    (7.5, 11.5, 'High Travel Time'),
]
for xmin, xmax, label in group_spans:
    ax.axvspan(xmin, xmax, color='grey', alpha=0.1, zorder=0)
    ax.text((xmin + xmax) / 2, ax.get_ylim()[1]*0.96, label,
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.tick_params(axis='x', length=0)
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

ax.grid(False)
sns.despine()
handles, labels = ax.get_legend_handles_labels()
mapped_labels = [method_map.get(l, l) for l in labels]

# legend = ax.legend(handles, mapped_labels, title='Method')
# remove legend
_current_legend = ax.get_legend()
if _current_legend is not None:
    _current_legend.remove()

plt.tight_layout()
plt.savefig(methods.file_path('cost_comparison_compiled.png', dir_name='figures'))
plt.show()

# Average Obeserved n individual Plot

In [ ]:
for row in observed_n_results_df.itertuples():
    
    test_name = row._1
    methods_list = list(observed_n_results_df.columns)[1:]
    
    if test_name == "Bicycle Data Test" or test_name == "Prof Data Test":
        methods_list = ['PPO', 'PTO_Var_n']
        
    method_list = [m for m in methods_list if not math.isnan(getattr(row, m))]
    
    observed_ns = [getattr(row, col) for col in methods_list]
    
    
    plt.figure(figsize=(10, 6))
    
    short_labels = [method_map[m] for m in methods_list]
    
    colors = [color_map[m] for m in methods_list]

   
    ax = sns.barplot(x=short_labels, y=observed_ns, palette=colors, edgecolor='black', linewidth=0.6)
    ax.set_ylabel('Average n')
    ax.set_xlabel('Method')
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.grid(False)
    
    sns.despine()
   
   
    for i, v in enumerate(observed_ns):
        ax.text(i, v + 0.05, f"{v:.3f}", ha='center', va='bottom')
    
    plt.tight_layout()
    
    config = sim_tests.get(test_name)
    
    file_name = f'observed_n_comparison_{test_name.replace(" ", "_").lower()}'
    plt.savefig(methods.file_path(f'{file_name}.png', dir_name='figures'))
    json.dump(config, open(methods.file_path(f'{file_name}.json', dir_name='figures'), 'w'), indent=4)
    plt.show()

## Obeserved n Compiled plot

In [ ]:
# obersved n compiled plot

# Preserve 'Test Name' so it can be used on x-axis
observed_n_results_df_melted = observed_n_results_df.melt(
	id_vars='Test Name',
	var_name='Method',
	value_name='Average n'
)
display(observed_n_results_df_melted.head())
observed_n_results_df_melted = observed_n_results_df_melted[observed_n_results_df_melted['Test Name'].str.contains('with', na=False)]
sns.set_theme(style='whitegrid', context='paper', font_scale=1.5, rc={"axes.labelweight": "bold"})
plt.figure(figsize=(10, 6))
ax = sns.barplot(
    data=observed_n_results_df_melted,
    x='Test Name',
    y='Average n',
    hue='Method',
    palette=color_map,
    edgecolor='black',
    linewidth=0.6
)

ax.set_ylabel('Average n', fontdict={'size': 14})
ax.set_xlabel('Simulated Gamma Tests', fontdict={'size': 14})
ax.set_xticklabels([
    r'low $\alpha$',
    r'mid $\alpha$',
    r'high $\alpha$',
    r'high $\beta$',  
] * 4)

# highlight groups: 1-4 uniform, 5-8 low, 9-12 high
group_spans = [
    (-0.5, 3.5, 'Uniform Travel Time'),
    (3.5, 7.5, 'Low Travel Time'),
    (7.5, 11.5, 'High Travel Time'),
]
for xmin, xmax, label in group_spans:
    ax.axvspan(xmin, xmax, color='grey', alpha=0.1, zorder=0)
    ax.text((xmin + xmax) / 2, ax.get_ylim()[1]*0.96, label,
            ha='center', va='bottom', fontsize=12, fontweight='bold')



ax.tick_params(axis='x', length=0)
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

ax.grid(False)
sns.despine()
handles, labels = ax.get_legend_handles_labels()
mapped_labels = [method_map.get(l, l) for l in labels]

# legend = ax.legend(handles, mapped_labels, title='Method')
# remove legend
_current_legend = ax.get_legend()
if _current_legend is not None:
    _current_legend.remove()

plt.tight_layout()

plt.savefig(methods.file_path('observed_n_comparison_compiled.png', dir_name='figures'))
plt.show()



## u* Comparision with alpha and beta

In [ ]:
# u* vs alpha plot
config = {
    'k': 10,
    'alpha_range': np.arange(0.1, 5, 0.1).tolist(),
    'beta': 2,
    'h': 0.5,
    'c': 25,
}
u_values = [methods.get_u_star_binary_fast(alpha, config['beta'], config['k'], config['h'], config['c']) for alpha in config['alpha_range']]

# plot u* vs alpha
plt.figure(figsize=(10, 6))
plt.plot(config['alpha_range'], u_values, marker='o')
plt.xlabel(r"$\alpha$")
plt.ylabel(r"$u_k^{*}$")
plt.grid()
plt.tight_layout()
sns.despine()
file_name = 'u_star_vs_alpha'
json.dump(config, open(methods.file_path(f'{file_name}.json', dir_name='figures'), 'w'), indent=4)
plt.savefig(methods.file_path('u_star_vs_alpha.png', dir_name='figures'))
plt.show()


# u* vs beta plot
config = {
    'k': 10,
    'alpha': 2,
    'beta_range': np.arange(0.1, 5, 0.1).tolist(),
    'h': 0.5,
    'c': 25,
}
u_values = [methods.get_u_star_binary_fast(config['alpha'], beta, config['k'], config['h'], config['c']) for beta in config['beta_range']]
# plot u* vs beta
plt.figure(figsize=(10, 6))
plt.plot(config['beta_range'], u_values, marker='o')
plt.xlabel(r"$\beta$")
plt.ylabel(r"$u_k^{*}$")
plt.grid()
plt.tight_layout()
sns.despine()
file_name = 'u_star_vs_beta'
json.dump(config, open(methods.file_path(f'{file_name}.json', dir_name='figures'), 'w'), indent=4)
plt.savefig(methods.file_path('u_star_vs_beta.png', dir_name='figures'))
plt.show()


## Plot travel time distributions

In [ ]:
for test_name, config in sim_tests.items():
    
    test_df_path = generate_data.generate(config=config, row_count=10_000)
    test_df = pd.DataFrame()
    with open(test_df_path, 'rb') as f:
        test_df =pickle.load(f)
        
    travel_times = test_df['travel_time'].values
    interval_sum = list(map(sum, test_df['intervals']))
    
    zip_data = list(zip(travel_times, interval_sum))
    
    # ratio of travel time to interval sum
    ratios = [tt / isum if isum > 0 else 0 for tt, isum in zip_data]
    plt.figure(figsize=(10, 6))
    sns.histplot(ratios, bins=50, kde=True)
    plt.xlabel('Travel Time / Sum of Intervals')
    plt.ylabel('Frequency')
    plt.title(f'Travel Time to Interval Sum Ratio Distribution\n{test_name}')
    plt.grid()
    sns.despine()
    plt.tight_layout()
    plt.show()
    